In [13]:
import numpy as np
import matplotlib.pyplot as plt
import experiment
import interpret
import cosmology
import consensus
import dill
import copy
import yaml

ModuleNotFoundError: No module named 'dill'

In [2]:
class Configuration():
    def __init__(self, config_file = None):
        self.config_dict = self.read(config_file)
        pass

    def read(self, config):
        """
        Reads config file into a dictionary and returns it.

        Args:
            config (str): Name of config file.

        Returns:
            config_dict (dict): Dictionary containing config information.
        """

        with open(config, 'r') as config_file_obj:
            config_dict = yaml.safe_load(config_file_obj)

        return config_dict


    def check_configuration(self,config):
        '''
        Checks that the configuration file makes sense, is compliant.
        '''
        necessary_params = ['consensus']#, 'experiment', 'interpreter']
        pass

In [94]:
times_B = np.linspace(0,10,500)
np.savetxt('times_B.txt',times_B)

In [17]:
times_A = np.linspace(2.,8.,500)
np.savetxt('times_A.txt',times_A)

In [3]:
test = Configuration(config_file='example.yaml')
consensus_sim = test.config_dict['consensus']
experiments = test.config_dict['experiments']

In [97]:
experiments['experiment A']['experimental parameters']['times description']

'some numbers to test (was np.linspace(2.,8.,500))'

In [98]:
experiments['experiment A']['class name']

'SimplePendulumExperiment'

In [99]:
experiments['experiment A']['experimental parameters'].values()

dict_values([[0.0, 0.1, 0.25, 0.5, 0.66, 0.9, 1.0], None, 'some numbers to test (was np.linspace(2.,8.,500))'])

In [100]:
[experiments[key]['noise_parameters'] for key in experiments.keys()]

[[0.03], [0.1]]

In [101]:
[np.array(experiments[key]['noise_parameters']) for key in experiments.keys()]

[array([0.03]), array([0.1])]

In [131]:
# new idea: don't define keys we're not using
times_keys = [experiments[key]['experimental parameters'].keys() for key in experiments.keys()]
times_keys

[dict_keys(['times', 'times description']),
 dict_keys(['times file', 'times description'])]

In [132]:
experimental_parameters = [np.array(experiments[key]['experimental parameters']['times']) if 'times' in experiments[key]['experimental parameters'].keys()
else np.loadtxt(experiments[key]['experimental parameters']['times file']) for key in experiments.keys()]

In [133]:
experimental_parameters[0]

array([0.  , 0.1 , 0.25, 0.5 , 0.66, 0.9 , 1.  ])

In [134]:
len(experimental_parameters[1])

500

In [4]:
# make it a dictionary
experimental_parameters = [{'times': np.array(experiments[key]['experimental parameters']['times'])} if 'times' in experiments[key]['experimental parameters'].keys()
        else {'times': np.loadtxt(experiments[key]['experimental parameters']['times file'])} for key in experiments.keys()]

In [5]:
type(experimental_parameters[1])

dict

In [6]:
experimental_parameters[0]

{'times': array([0.  , 0.1 , 0.25, 0.5 , 0.66, 0.9 , 1.  ])}

In [8]:
interpreter_names = [experiments[key]['interpreter']['class name'] for key in experiments.keys()]

In [9]:
interpreter_names

['SimplePendulumExperimentInterpreter', 'SimplePendulumExperimentInterpreter']

In [10]:
len(experimental_parametersmeters)

2

In [15]:
consensus_names = consensus_sim.keys()

In [16]:
for name in consensus_names:
    kwargs = consensus_sim[name]
    print(kwargs)

{'patience': 5, 'chi2_dof_threshold': 1.25}
{'patience': 5, 'chi2_dof_threshold': 1.25, 'tolerance': 2}
{'patience': 5, 'chi2_dof_threshold': 1.25, 'tolerance': 2}
{'patience': 5, 'chi2_dof_threshold': 1.25}
{'patience': 5, 'chi2_dof_threshold': 1.25}
{'chi2_dof_threshold': 1.25}
{'chi2_dof_threshold': 1.25}
